В дополнительных заданиях можно реализовывать логику только частично, тогда
баллы будут снижены. На оценку за каждое задание также влияет корректность шагов и
качество кода.
Задания под * не являются обязательными, но будут учитываться при отборе
участников.

Рекомендации к оборудованию и ПО
• Для эффективной работы тебе потребуется ПК / ноутбук.
• Если с работой программы возникнут проблемы, пиши на shift_support@cft.ru
• Установи и подготовь к работе ПО: Docker, Jupyter Notebook, IDE PyCharm, IntelliJIDEA

Задание
Необходимо реализовать ETL1 пайплайн (В данном случае приложение на Scala, Java
или Python), которое загружает исходные данные, трансформирует их, а затем выгружает)
для выгрузки данных прогнозов погоды.
Для получения данных будет использован https://open-meteo.com/

Пример запроса данных по API open-meteo:
https://api.openmeteo.com/v1/forecast?latitude=55.0344&longitude=82.9434&daily=sunrise,sunset,daylight_duration&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,temperature_80m,temperature_120m,wind_speed_10m,wind_speed_80m,wind_direction_10m,wind_direction_80m,visibility,evapotranspiration,weather_code,soil_temperature_0cm,soil_temperature_6cm,rain,showers,snowfall&timezone=auto&timeformat=unixtime&wind_speed_unit=kn&temperature_unit=fahrenheit&precipitation_unit=inch&start_date=2025-05-16&end_date=2025-05-30

Описание данных JSON в приложении, таблица "Описание данных JSON"

Задание 1: Наивная реализация
Реализовать приложение, которое будет делать следующее:
1. считывать json в формате API open-meteo, json взять из запроса в
"Пример запроса данных по API open-meteo"
2. Выполнять трансформации данных, которые на Ваш взгляд необходимы для
получения данных в формате для выгрузки - т.е в формате, как в таблице
"Итоговая таблица" из приложения.
3. Сохранить данные в csv. Поля, необходимые в csv и их описание представлены в
таблице "Итоговая таблица" из приложения.

In [41]:
import requests
import json

base_url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 55.0344,
    "longitude": 82.9434,
    "daily": "sunrise,sunset,daylight_duration",
    "hourly": (        
        "temperature_2m,relative_humidity_2m,dew_point_2m,"
        "apparent_temperature,temperature_80m,temperature_120m,"
        "wind_speed_10m,wind_speed_80m,wind_direction_10m,"
        "wind_direction_80m,visibility,evapotranspiration,"
        "weather_code,soil_temperature_0cm,soil_temperature_6cm,"
        "rain,showers,snowfall"
        ),
    "timezone": "auto",
    "start_date": "2025-05-16",
    "end_date": "2025-05-30"
}

response = requests.get(base_url, params=params)
meteo_data = response.json()
print(meteo_data)  

with open('weather_data.json', 'w', encoding='utf-8') as f:
    json.dump(meteo_data, f, ensure_ascii=False, indent=4)



print("Status Code:", response.status_code)  
print("Response Text:", response.text[:200])  


{'latitude': 55.0, 'longitude': 83.0, 'generationtime_ms': 16.646981239318848, 'utc_offset_seconds': 25200, 'timezone': 'Asia/Novosibirsk', 'timezone_abbreviation': 'GMT+7', 'elevation': 135.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C', 'relative_humidity_2m': '%', 'dew_point_2m': '°C', 'apparent_temperature': '°C', 'temperature_80m': '°C', 'temperature_120m': '°C', 'wind_speed_10m': 'km/h', 'wind_speed_80m': 'km/h', 'wind_direction_10m': '°', 'wind_direction_80m': '°', 'visibility': 'm', 'evapotranspiration': 'mm', 'weather_code': 'wmo code', 'soil_temperature_0cm': '°C', 'soil_temperature_6cm': '°C', 'rain': 'mm', 'showers': 'mm', 'snowfall': 'cm'}, 'hourly': {'time': ['2025-05-16T00:00', '2025-05-16T01:00', '2025-05-16T02:00', '2025-05-16T03:00', '2025-05-16T04:00', '2025-05-16T05:00', '2025-05-16T06:00', '2025-05-16T07:00', '2025-05-16T08:00', '2025-05-16T09:00', '2025-05-16T10:00', '2025-05-16T11:00', '2025-05-16T12:00', '2025-05-16T13:00', '2025-05-16T14:00', '20

Вычисляем среднюю температуру  avg_temperature_2m_24h

In [42]:
import json

with open('weather_data.json', 'r', encoding='utf-8') as f:
    meteo_data = json.load(f)




In [ ]:
import numpy as np # используется для более эффективного разбиения и вычисления avg

def calculating_avg_24(data, chunk_size):
    '''
    Функция для разбиения списка на подсписки длины chunk 
    и вычисления среднего значения
    '''
    
    list_avg_values = []

    arr = np.array(data)
    chunks = np.array_split(arr, len(data) // chunk_size) # Разбиение на чанки

    for chunk in chunks:
        if len(chunk) >= (chunk_size - 1):
            avg = round(chunk.mean(), 1) # Подсчёт среднего сточностью 1
            list_avg_values.append(avg)
            #print(list_avg_values)
        else:
            print(f'len_chunk = {len(chunk)}')
    return list_avg_values
        

[np.float64(9.7), np.float64(11.6), np.float64(13.0), np.float64(13.6), np.float64(16.7), np.float64(19.5), np.float64(21.1), np.float64(22.4), np.float64(16.8), np.float64(12.2), np.float64(9.1), np.float64(4.3), np.float64(5.0), np.float64(7.7), np.float64(13.5)]


In [ ]:
final_list_24 = (
    "temperature_2m", "relative_humidity_2m", "dew_point_2m",
    "apparent_temperature", "temperature_80m", "temperature_120m",
    "wind_speed_10m", "wind_speed_80m","visibility", 
    "rain", "showers", "snowfall"    
) # Коортеж для более эффективного исползования памяти

#print(final_list)

for parametr in final_list_24:
    final_param = calculating_avg_24(meteo_data['hourly'][parametr], 24)
    print(final_param)

#print(type(final_list_24))

[np.float64(9.7), np.float64(11.6), np.float64(13.0), np.float64(13.6), np.float64(16.7), np.float64(19.5), np.float64(21.1), np.float64(22.4), np.float64(16.8), np.float64(12.2), np.float64(9.1), np.float64(4.3), np.float64(5.0), np.float64(7.7), np.float64(13.5)]
[np.float64(61.3), np.float64(79.7), np.float64(80.3), np.float64(54.8), np.float64(47.3), np.float64(46.8), np.float64(43.3), np.float64(46.9), np.float64(64.0), np.float64(57.7), np.float64(58.7), np.float64(57.8), np.float64(68.5), np.float64(58.9), np.float64(61.0)]
[np.float64(2.0), np.float64(8.0), np.float64(9.6), np.float64(3.6), np.float64(4.7), np.float64(6.9), np.float64(7.1), np.float64(9.0), np.float64(9.8), np.float64(3.7), np.float64(1.2), np.float64(-3.7), np.float64(-0.5), np.float64(-0.8), np.float64(5.7)]
[np.float64(6.7), np.float64(10.4), np.float64(11.7), np.float64(10.4), np.float64(14.0), np.float64(17.6), np.float64(19.4), np.float64(21.1), np.float64(15.0), np.float64(9.8), np.float64(5.6), np.float

In [45]:
from datetime import datetime
import numpy as np

def calculate_daylight_avg_param(meteo_data, meteo_param):
    """
    Вычисляет среднее значение параметра в светлое время суток для каждого дня.
    
    Алгоритм:
    1) Группирует часовые данные по дням
    2) Для каждого дня определяет временной интервал между восходом и закатом
    3) Выбирает значения параметра, попадающие в этот интервал
    4) Рассчитывает среднее значение для светового дня
    """
    
    # Преобразуем строки времени в объекты datetime
    hourly_times = [datetime.fromisoformat(t) for t in meteo_data["hourly"]["time"]]
    hourly_values = meteo_data["hourly"][meteo_param]
    
    # Создаем словарь для группировки по дням
    from collections import defaultdict
    daily_hourly_data = defaultdict(list)
    
    # Группируем данные по дням
    for time, value in zip(hourly_times, hourly_values):
        date_key = time.date()  # Используем только дату без времени
        daily_hourly_data[date_key].append((time.time(), value))
    
    # Получаем даты из daily данных
    daily_dates = [datetime.fromisoformat(t).date() for t in meteo_data["daily"]["time"]]
    daily_sunrises = [datetime.fromisoformat(s).time() for s in meteo_data["daily"]["sunrise"]]
    daily_sunsets = [datetime.fromisoformat(s).time() for s in meteo_data["daily"]["sunset"]]
    
    daylight_avg_values = []
    
    # Обрабатываем каждый день 
    for date, sunrise, sunset in zip(daily_dates, daily_sunrises, daily_sunsets):
        if date not in daily_hourly_data:
            daylight_avg_values.append(None)
            continue
            
        day_values = []
        
        # Проверяем каждый час для текущего дня
        for time, value in daily_hourly_data[date]:
            # Если текущее время попадает в интервал между восходом и закатом
            if sunrise <= time <= sunset:
                day_values.append(value)
        
        # Вычисляем среднее для светового дня
        if day_values:
            avg_value = np.mean(day_values)
            daylight_avg_values.append(round(avg_value, 1))
        else:
            daylight_avg_values.append(None)
    
    return daylight_avg_values



In [47]:
final_list_daylight = (
    "temperature_2m", "relative_humidity_2m", "dew_point_2m",
    "apparent_temperature", "temperature_80m", "temperature_120m",
    "wind_speed_10m", "wind_speed_80m","visibility"   
)

for parametr in final_list_daylight:
    final_param = calculate_daylight_avg_param(meteo_data, parametr)
    print(final_param)

[np.float64(9.5), np.float64(13.0), np.float64(13.4), np.float64(14.4), np.float64(18.6), np.float64(21.5), np.float64(22.6), np.float64(24.8), np.float64(16.6), np.float64(13.0), np.float64(9.6), np.float64(4.8), np.float64(5.4), np.float64(8.8), np.float64(14.8)]
[np.float64(67.2), np.float64(75.2), np.float64(77.4), np.float64(46.5), np.float64(42.4), np.float64(40.7), np.float64(38.4), np.float64(40.0), np.float64(63.1), np.float64(52.8), np.float64(55.4), np.float64(53.6), np.float64(70.9), np.float64(52.8), np.float64(57.1)]
[np.float64(3.5), np.float64(8.6), np.float64(9.4), np.float64(2.2), np.float64(4.7), np.float64(6.6), np.float64(6.7), np.float64(8.7), np.float64(9.4), np.float64(3.2), np.float64(0.9), np.float64(-4.2), np.float64(0.3), np.float64(-1.4), np.float64(5.9)]
[np.float64(6.6), np.float64(11.8), np.float64(12.1), np.float64(10.8), np.float64(15.6), np.float64(19.2), np.float64(20.8), np.float64(23.1), np.float64(14.3), np.float64(10.6), np.float64(5.8), np.float